In [1]:
import pandas as pd
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
import matplotlib as plt
import numpy as np
import os
import xlrd
import datetime

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
# 输入原始数据
df_aoi=pd.read_csv('d:\\user\\01389648\\desktop\\PythonWorkSpace\\2020算法_数据分析挑战赛\\算法大赛\\aoi信息.csv')
df_xiaoge=pd.read_csv('d:\\user\\01389648\\desktop\\PythonWorkSpace\\2020算法_数据分析挑战赛\\算法大赛\\小哥列表.csv')
df_jianliang=pd.read_csv('d:\\user\\01389648\\desktop\\PythonWorkSpace\\2020算法_数据分析挑战赛\\算法大赛\\比赛-件量-train.csv')
df_workday=pd.read_csv('d:\\user\\01389648\\desktop\\PythonWorkSpace\\2020算法_数据分析挑战赛\\算法大赛\\workdays2020.csv')

In [3]:
df_jianliang=df_jianliang.groupby(['zone_id','date_str',]).agg({'pai_num':sum,'shou_num':sum}).reset_index()
df_jianliang

zone_id  date_str  pai_num  shou_num
0     zone_0  20200301      481      1062
1     zone_0  20200302      478       470
2     zone_0  20200303      603       476
3     zone_0  20200304      565       437
4     zone_0  20200305      545       440
...      ...       ...      ...       ...
4873  zone_9  20200806      449       242
4874  zone_9  20200807      474       259
4875  zone_9  20200808      364        91
4876  zone_9  20200809      334       100
4877  zone_9  20200810      457       252

[4878 rows x 4 columns]

In [4]:
df_jianliang['date_str']=df_jianliang['date_str'].astype(str)
df_jianliang['weekday']=df_jianliang['date_str'].apply(lambda x: datetime.datetime.strptime(x,'%Y%m%d').weekday()+1)
df_jianliang

zone_id  date_str  pai_num  shou_num  weekday
0     zone_0  20200301      481      1062        7
1     zone_0  20200302      478       470        1
2     zone_0  20200303      603       476        2
3     zone_0  20200304      565       437        3
4     zone_0  20200305      545       440        4
...      ...       ...      ...       ...      ...
4873  zone_9  20200806      449       242        4
4874  zone_9  20200807      474       259        5
4875  zone_9  20200808      364        91        6
4876  zone_9  20200809      334       100        7
4877  zone_9  20200810      457       252        1

[4878 rows x 5 columns]

In [5]:
df_jianliang['ds']=pd.to_datetime(df_jianliang['date_str'])

In [6]:
df_workday['date']=df_workday['date'].astype(str)
df_workday['dt']=pd.to_datetime(df_workday['date'])
df_workday

date  workdays         dt
0      2020/3/1         3 2020-03-01
1      2020/3/2         0 2020-03-02
2      2020/3/3         0 2020-03-03
3      2020/3/4         0 2020-03-04
4      2020/3/5         0 2020-03-05
..          ...       ...        ...
301  2020/12/27         3 2020-12-27
302  2020/12/28         0 2020-12-28
303  2020/12/29         0 2020-12-29
304  2020/12/30         0 2020-12-30
305  2020/12/31         0 2020-12-31

[306 rows x 3 columns]

In [7]:
df_jianliang=df_jianliang.merge(df_workday, left_on='ds', right_on='dt')
df_jianliang

zone_id  date_str  pai_num  shou_num  weekday         ds       date  \
0      zone_0  20200301      481      1062        7 2020-03-01   2020/3/1   
1      zone_1  20200301      235       290        7 2020-03-01   2020/3/1   
2     zone_10  20200301      229       482        7 2020-03-01   2020/3/1   
3     zone_11  20200301      248       169        7 2020-03-01   2020/3/1   
4     zone_12  20200301      523       209        7 2020-03-01   2020/3/1   
...       ...       ...      ...       ...      ...        ...        ...   
4873   zone_5  20200810      462       424        1 2020-08-10  2020/8/10   
4874   zone_6  20200810      350       224        1 2020-08-10  2020/8/10   
4875   zone_7  20200810      358       195        1 2020-08-10  2020/8/10   
4876   zone_8  20200810      502       336        1 2020-08-10  2020/8/10   
4877   zone_9  20200810      457       252        1 2020-08-10  2020/8/10   

      workdays         dt  
0            3 2020-03-01  
1            3 2020-03-01  
2            3 2020-03-01  
3            3 2020-03-01  
4            3 2020-03-01  
...        ...        ...  
4873         0 2020-08-10  
4874         0 2020-08-10  
4875         0 2020-08-10  
4876         0 2020-08-10  
4877         0 2020-08-10  

[4878 rows x 9 columns]

In [8]:
df_shou=df_jianliang[['ds','shou_num','workdays','weekday','zone_id']]
df_shou.rename(columns={'shou_num':'y'},inplace=True)
df_shou

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


ds     y  workdays  weekday  zone_id
0    2020-03-01  1062         3        7   zone_0
1    2020-03-01   290         3        7   zone_1
2    2020-03-01   482         3        7  zone_10
3    2020-03-01   169         3        7  zone_11
4    2020-03-01   209         3        7  zone_12
...         ...   ...       ...      ...      ...
4873 2020-08-10   424         0        1   zone_5
4874 2020-08-10   224         0        1   zone_6
4875 2020-08-10   195         0        1   zone_7
4876 2020-08-10   336         0        1   zone_8
4877 2020-08-10   252         0        1   zone_9

[4878 rows x 5 columns]

In [9]:
df_pai=df_jianliang[['ds','pai_num','workdays','weekday','zone_id']]
df_pai

ds  pai_num  workdays  weekday  zone_id
0    2020-03-01      481         3        7   zone_0
1    2020-03-01      235         3        7   zone_1
2    2020-03-01      229         3        7  zone_10
3    2020-03-01      248         3        7  zone_11
4    2020-03-01      523         3        7  zone_12
...         ...      ...       ...      ...      ...
4873 2020-08-10      462         0        1   zone_5
4874 2020-08-10      350         0        1   zone_6
4875 2020-08-10      358         0        1   zone_7
4876 2020-08-10      502         0        1   zone_8
4877 2020-08-10      457         0        1   zone_9

[4878 rows x 5 columns]

In [10]:
zone=[i for i in df_shou['zone_id'].unique()]
len(zone)

30

In [11]:
zone

['zone_0',
 'zone_1',
 'zone_10',
 'zone_11',
 'zone_12',
 'zone_13',
 'zone_14',
 'zone_15',
 'zone_16',
 'zone_17',
 'zone_18',
 'zone_19',
 'zone_2',
 'zone_20',
 'zone_21',
 'zone_22',
 'zone_23',
 'zone_24',
 'zone_25',
 'zone_26',
 'zone_27',
 'zone_28',
 'zone_29',
 'zone_3',
 'zone_4',
 'zone_5',
 'zone_6',
 'zone_7',
 'zone_8',
 'zone_9']

# 收件预测

In [12]:
def zone_shou_predict(df, i):
    
    #实例化Prophet算法
    m=Prophet(changepoint_prior_scale=0.5)
    #m.add_country_holidays(country_name='CN')
    m.add_regressor('workdays')
    m.fit(df)
    
    #扩展预测区间且进行预测
    future=pd.DataFrame()
    forecast=pd.DataFrame()
    future=m.make_future_dataframe(periods=21)
    future=future.merge(df_workday, left_on='ds', right_on='dt')[['ds','workdays']]
    forecast=m.predict(future)
    forecast['zone_id']=i
    forecast[['ds','yhat','yhat_lower','yhat_upper','zone_id']].tail(21).to_csv('d:\\user\\01389648\\desktop\\PythonWorkSpace\\2020算法_数据分析挑战赛\\算法大赛\\各区域收件预测_output.csv', mode='a')
    
    #输出CV
    
    df_cv=pd.DataFrame()
    df_cv=cross_validation(m, initial='120 days', period='7 days', horizon='7 days')
    df_p=pd.DataFrame()
    df_p=performance_metrics(df_cv, rolling_window=0)
    df_p['zone_id']= i
    df_p.to_csv('d:\\user\\01389648\\desktop\\PythonWorkSpace\\2020算法_数据分析挑战赛\\算法大赛\\各城市收件_mape.csv', mode='a')
    

    return

In [13]:
for i in zone:
    df=pd.DataFrame()
    df=df_shou.loc[(df_shou['zone_id']==i), :][['ds','y']]
    df=df.merge(df_workday, left_on ='ds',right_on='dt')[['ds','y','workdays']]
    zone_shou_predict(df, i)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 6 forecasts with cutoffs between 2020-06-29 00:00:00 and 2020-08-03 00:00:00
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 6 forecasts with cutoffs between 2020-06-29 00:00:00 and 2020-08-03 00:00:00
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 6 forecasts with cutoffs between 2020-06-29 00:00:00 and 2020-08-03 00:00:00
INFO:fbprophet:Disabling yearly s

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 6 forecasts with cutoffs between 2020-06-29 00:00:00 and 2020-08-03 00:00:00
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 6 forecasts with cutoffs between 2020-06-29 00:00:00 and 2020-08-03 00:00:00
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Making 6 forecasts with cutoffs between 2020-06-29 00:00:00 and 2020-08-03 00:00:00
